In [ ]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path

class LibrosaVocalRemover:
    def __init__(self):
        self.sr = None
        self.audio = None

    def load_audio(self, file_path):
        """Загрузка аудиофайла"""
        try:
            print(f"📁 Загрузка аудио: {file_path}")
            self.audio, self.sr = librosa.load(file_path, sr=None, mono=False)
            print(f"✅ Аудио загружено: {self.audio.shape}, Sample Rate: {self.sr}")
            return True
        except Exception as e:
            print(f"❌ Ошибка загрузки аудио: {e}")
            return False

    def spectral_subtraction(self, output_path="instrumental.wav"):
        """
        Основной метод - спектральное вычитание для стерео файлов
        """
        if self.audio is None:
            print("❌ Сначала загрузите аудио!")
            return None

        print("🎵 Применение спектрального вычитания...")

        try:
            if len(self.audio.shape) > 1:  # Стерео
                print("🔊 Обработка стерео аудио...")

                # Получаем левый и правый каналы
                left = self.audio[0]
                right = self.audio[1]

                # Спектральное вычитание - вокал обычно в центре (одинаковый в обоих каналах)
                instrumental = left - right

                # Улучшаем качество
                instrumental = self._enhance_audio(instrumental)

                # Сохраняем как моно
                sf.write(output_path, instrumental, self.sr)

            else:  # Моно
                print("🔊 Обработка моно аудио...")
                instrumental = self._process_mono_audio(self.audio)
                sf.write(output_path, instrumental, self.sr)

            print(f"✅ Инструментал сохранен: {output_path}")
            return output_path

        except Exception as e:
            print(f"❌ Ошибка: {e}")
            return None

    def advanced_spectral_method(self, output_path="instrumental_advanced.wav"):
        """
        Продвинутый метод с STFT обработкой
        """
        if self.audio is None:
            print("❌ Сначала загрузите аудио!")
            return None

        print("🎵 Применение продвинутого спектрального метода...")

        try:
            if len(self.audio.shape) > 1:  # Стерео
                # Обрабатываем оба канала и комбинируем
                left_processed = self._process_channel(self.audio[0])
                right_processed = self._process_channel(self.audio[1])

                # Комбинируем результаты
                instrumental = (left_processed + right_processed) / 2
            else:
                instrumental = self._process_channel(self.audio)

            instrumental = self._enhance_audio(instrumental)
            sf.write(output_path, instrumental, self.sr)

            print(f"✅ Продвинутый инструментал сохранен: {output_path}")
            return output_path

        except Exception as e:
            print(f"❌ Ошибка: {e}")
            return None

    def _process_channel(self, channel):
        """Обработка одного аудио канала"""
        # STFT преобразование
        stft = librosa.stft(channel, n_fft=2048, hop_length=512)
        magnitude = np.abs(stft)
        phase = np.angle(stft)

        # Медианная фильтрация для удаления вокальных компонентов
        filtered_magnitude = signal.medfilt(magnitude, kernel_size=(1, 5))

        # Восстанавливаем аудио
        processed_stft = filtered_magnitude * np.exp(1j * phase)
        reconstructed = librosa.istft(processed_stft, hop_length=512)

        return reconstructed

    def _process_mono_audio(self, mono_audio):
        """Обработка моно аудио"""
        # Применяем полосовой фильтр для выделения инструментальных частот
        nyquist = self.sr / 2

        # Фильтр для инструментальных частот (исключаем основные вокальные частоты)
        lowcut = 100 / nyquist   # 100 Hz
        highcut = 4000 / nyquist # 4000 Hz

        # Создаем полосовой фильтр Баттерворта
        b, a = signal.butter(6, [lowcut, highcut], btype='band')
        filtered = signal.filtfilt(b, a, mono_audio)

        return filtered

    def _enhance_audio(self, audio):
        """Улучшение качества аудио"""
        # Нормализация
        max_val = np.max(np.abs(audio))
        if max_val > 0:
            audio = audio / max_val

        # Легкая компрессия
        audio = np.tanh(audio * 0.8)

        return audio

    def create_instrumental(self, input_file, method='simple'):
        """
        Основная функция для создания инструментала
        """
        if not self.load_audio(input_file):
            return None

        base_name = Path(input_file).stem

        if method == 'simple':
            output_path = f"{base_name}_instrumental.wav"
            return self.spectral_subtraction(output_path)
        elif method == 'advanced':
            output_path = f"{base_name}_instrumental_advanced.wav"
            return self.advanced_spectral_method(output_path)
        else:
            print("❌ Неизвестный метод. Используйте 'simple' или 'advanced'")
            return None

# Простая функция для быстрого использования
def create_instrumental_simple(input_file, output_file=None):
    """
    Простая функция для создания инструментала
    """
    if output_file is None:
        base_name = Path(input_file).stem
        output_file = f"{base_name}_instrumental.wav"

    remover = LibrosaVocalRemover()

    if remover.load_audio(input_file):
        return remover.spectral_subtraction(output_file)
    else:
        return None

def batch_process_folder(folder_path, output_folder="instrumentals"):
    """
    Пакетная обработка всех аудиофайлов в папке
    """
    audio_extensions = ['.mp3', '.wav', '.flac', '.m4a', '.ogg']

    # Создаем выходную папку
    os.makedirs(output_folder, exist_ok=True)

    # Ищем аудиофайлы
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(Path(folder_path).glob(f"*{ext}"))
        audio_files.extend(Path(folder_path).glob(f"*{ext.upper()}"))

    print(f"🎵 Найдено {len(audio_files)} аудиофайлов")

    results = []
    remover = LibrosaVocalRemover()

    for audio_file in audio_files:
        print(f"\n🔊 Обработка: {audio_file.name}")

        output_path = os.path.join(output_folder, f"{audio_file.stem}_instrumental.wav")

        if remover.load_audio(str(audio_file)):
            result = remover.spectral_subtraction(output_path)
            if result:
                results.append(result)

    print(f"\n✅ Готово! Создано {len(results)} инструменталов")
    return results

In [ ]:
create_instrumental_simple('/content/Kanye_West_-_I_Wonder_(Rilds.com).mp3')

📁 Загрузка аудио: /content/Kanye_West_-_I_Wonder_(Rilds.com).mp3
✅ Аудио загружено: (2, 10737839), Sample Rate: 44100
🎵 Применение спектрального вычитания...
🔊 Обработка стерео аудио...
✅ Инструментал сохранен: Kanye_West_-_I_Wonder_(Rilds.com)_instrumental.wav


'Kanye_West_-_I_Wonder_(Rilds.com)_instrumental.wav'